In [1]:
df = spark.read.csv("s3://airportsparkcsvdata/Airports2.csv", header=True, inferSchema=True)
df.registerTempTable('df')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1629808063391_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Origin_airport: string (nullable = true)
 |-- Destination_airport: string (nullable = true)
 |-- Origin_city: string (nullable = true)
 |-- Destination_city: string (nullable = true)
 |-- Passengers: integer (nullable = true)
 |-- Seats: integer (nullable = true)
 |-- Flights: integer (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- Fly_date: string (nullable = true)
 |-- Origin_population: integer (nullable = true)
 |-- Destination_population: integer (nullable = true)
 |-- Org_airport_lat: string (nullable = true)
 |-- Org_airport_long: string (nullable = true)
 |-- Dest_airport_lat: string (nullable = true)
 |-- Dest_airport_long: string (nullable = true)

In [3]:
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+-------------------+--------------+----------------+------------------+-----------------+------------------+-----------------+----------+-----------------+----------------------+------------------+------------------+-----------------+------------------+
|summary|Origin_airport|Destination_airport|   Origin_city|Destination_city|        Passengers|            Seats|           Flights|         Distance|  Fly_date|Origin_population|Destination_population|   Org_airport_lat|  Org_airport_long| Dest_airport_lat| Dest_airport_long|
+-------+--------------+-------------------+--------------+----------------+------------------+-----------------+------------------+-----------------+----------+-----------------+----------------------+------------------+------------------+-----------------+------------------+
|  count|       3606803|            3606803|       3606803|         3606803|           3606803|          3606803|           3606803|          3606803|   3606803|     

In [4]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3606803

In [5]:
df.select("Origin_airport","Destination_airport","Passengers","Seats").show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------------+----------+-----+
|Origin_airport|Destination_airport|Passengers|Seats|
+--------------+-------------------+----------+-----+
|           MHK|                AMW|        21|   30|
|           EUG|                RDM|        41|  396|
|           EUG|                RDM|        88|  342|
|           EUG|                RDM|        11|   72|
|           MFR|                RDM|         0|   18|
|           MFR|                RDM|        11|   18|
|           MFR|                RDM|         2|   72|
|           MFR|                RDM|         7|   18|
|           MFR|                RDM|         7|   36|
|           SEA|                RDM|         8|   18|
|           SEA|                RDM|       453| 3128|
|           SEA|                RDM|       784| 2720|
|           SEA|                RDM|       749| 2992|
|           SEA|                RDM|        11|   18|
|           PDX|                RDM|       349|  851|
+--------------+------------

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import desc

airportAgg_DF = df.groupBy("Origin_airport").agg(F.sum("Passengers"))
airportAgg_DF.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+---------------+
|Origin_airport|sum(Passengers)|
+--------------+---------------+
|           BGM|        1876537|
|           VWD|              0|
|           GEG|       23872254|
|           OPF|           7534|
|           MYR|       10386029|
|           CNW|              0|
|           EAR|           2548|
|           MQT|         720004|
|           YKN|              0|
|           GWO|             45|
+--------------+---------------+
only showing top 10 rows

In [7]:
originAirports = sqlContext.sql("""select Origin_Airport, sum(Flights) as Flights 
                                    from df group by Origin_Airport order by sum(Flights) DESC limit 10""")
originAirports.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------+
|Origin_Airport|Flights|
+--------------+-------+
|           ORD|6908482|
|           ATL|6558015|
|           DFW|5994638|
|           LAX|4099901|
|           DTW|3452613|
|           PHX|3213108|
|           MSP|3204923|
|           IAH|3195062|
|           STL|3181102|
|           CLT|2840773|
+--------------+-------+

In [8]:
destinationAirports = sqlContext.sql("""select Destination_airport, sum(Passengers) as Passengers 
                                    from df group by Destination_airport order by sum(Passengers) DESC limit 10""")
destinationAirports.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+
|Destination_airport|Passengers|
+-------------------+----------+
|                ATL| 577954147|
|                ORD| 528648148|
|                DFW| 458322527|
|                LAX| 389476602|
|                PHX| 295580444|
|                LAS| 269145891|
|                DTW| 251467874|
|                MSP| 245774036|
|                SFO| 242283245|
|                IAH| 229105403|
+-------------------+----------+

In [9]:
MostFlightsByAirports = sqlContext.sql("""with destination as (select Destination_airport as Airport, sum(Flights) as Out_Flights 
                                    from df group by Destination_airport),
                                    origin as (select Origin_airport as Airport, sum(Flights) as In_Flights 
                                    from df group by Origin_airport)
                                    select origin.Airport, (destination.Out_Flights+origin.In_Flights) as Total_Flights
                                    from origin, destination 
                                    where origin.Airport = destination.Airport
                                    order by (origin.In_Flights + destination.Out_Flights) DESC
                                    limit 15""")
MostFlightsByAirports.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
|Airport|Total_Flights|
+-------+-------------+
|    ORD|     13804767|
|    ATL|     13102774|
|    DFW|     11982524|
|    LAX|      8196603|
|    DTW|      6900655|
|    PHX|      6421985|
|    MSP|      6405105|
|    IAH|      6391830|
|    STL|      6358741|
|    CLT|      5677880|
|    EWR|      5620104|
|    LGA|      5368743|
|    PHL|      5344278|
|    SEA|      5158456|
|    LAS|      5040914|
+-------+-------------+

In [10]:
distanceQuery = sqlContext.sql("""with table1 as 
                                    (select least(Origin_airport, Destination_airport) as Airport1, 
                                    greatest(Destination_airport, Origin_airport) as Airport2, 
                                    sum(Flights) as Flights,
                                    sum(Passengers) as Passengers,
                                    sum(Seats) as Seats
                                    from df
                                    group by least(Origin_airport, Destination_airport), greatest(Destination_airport, Origin_airport)
                                    order by 1,2)
                                    select t.*, (Passengers*100/Seats) as Occupancy_Rate
                                    from table1 t
                                    order by Flights DESC, Seats DESC, Passengers DESC, Occupancy_Rate DESC
                                    limit 15;""")
distanceQuery = distanceQuery.filter((col("Occupancy_Rate").isNotNull()) & (col("Occupancy_Rate")<=100.0))
distanceQuery.show(15)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+-------+----------+--------+------------------+
|Airport1|Airport2|Flights|Passengers|   Seats|    Occupancy_Rate|
+--------+--------+-------+----------+--------+------------------+
|     HNL|     OGG| 784873|  62109354|96640901| 64.26818599300931|
|     LAX|     SFO| 636449|  51119989|79405656|  64.3782717442697|
|     LAS|     LAX| 588151|  52511530|80532768| 65.20517213564546|
|     PDX|     SEA| 565707|  18475771|34650955| 53.31965886654495|
|     LAX|     PHX| 515093|  42695385|65619395| 65.06519147273455|
|     BOS|     LGA| 470737|  31242486|64897330| 48.14140427657039|
|     MSP|     ORD| 467514|  31301666|55325318|56.577471457100344|
|     LAS|     PHX| 460104|  42979048|64844100| 66.28058373853597|
|     DCA|     LGA| 439107|  29471657|60663368|   48.582295991215|
|     LAX|     SAN| 431076|  11686171|22820096|51.209999291852235|
|     LGA|     ORD| 424272|  39981416|59616532| 67.06431028225526|
|     DAL|     HOU| 408273|  35573141|53054549| 67.05012420329